<a href="https://colab.research.google.com/github/luciainnocenti/IncrementalLearning/blob/Lucia/TrainAndTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import GitHub repository

In [0]:
import os
import logging
import sys


In [2]:
if not os.path.isdir('./DatasetCIFAR'):
  !git clone -b Lucia https://github.com/luciainnocenti/IncrementalLearning.git
  !mv 'IncrementalLearning' 'DatasetCIFAR'

Cloning into 'IncrementalLearning'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 366 (delta 21), reused 0 (delta 0), pack-reused 330
Receiving objects: 100% (366/366), 275.79 KiB | 510.00 KiB/s, done.
Resolving deltas: 100% (220/220), done.


# Import packages

In [0]:
from DatasetCIFAR.data_set import Dataset 
from DatasetCIFAR import ResNet
from DatasetCIFAR import utils
from DatasetCIFAR import params
from torchvision import models
import torch.nn as nn
import torch
import torch.optim as optim
import torchvision
import numpy as np

from torchvision import transforms
from torch.utils.data import Subset, DataLoader
from torch.nn import functional as F
import random
random.seed(42)

# Define Network

In [0]:
resNet = ResNet.resnet32(num_classes=100)
resNet = resNet.to(params.DEVICE)

In [0]:
resnet_transformer = transforms.Compose([transforms.Resize(32), 
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
])


# Define DataSets

In [6]:
trainDS = Dataset(train=True, transform = resnet_transformer)
testDS = Dataset(train=False, transform = resnet_transformer)

Extracting data/cifar-100-python.tar.gz to data
Files already downloaded and verified


In [0]:
train_splits = trainDS.splits
test_splits = testDS.splits

# Useful plots

The function plotEpoch plots, at the end of each task, how accuracy and loss change during the training phase. It show

*   Validation and Training Accuracy
*   Validation and Training Loss

The function plotTask, for each task, how the accuracy on the validation set change when adding new tasks


In [0]:
def plotTask(pars_tasks):
  import numpy as np
  import matplotlib.pyplot as plt

  x_tasks =  np.linspace(10, 100, 10)

  plt.plot(x_tasks, pars_tasks, label=['Accuracy', 'Loss'])
  plt.xlabel("Epoch")
  plt.title('Accuracy over tasks')
  plt.legend(['Accuracy', 'Loss'])
  plt.grid(True)
  plt.show()



# Train and evaluation phase

In [0]:
pars_tasks = []
test_indexes = []

for task in range(0, 100, params.TASK_SIZE):
  pars_tasks.insert(task, 0)

for task in range(0, 100, params.TASK_SIZE):

  train_indexes = trainDS.__getIndexesGroups__(task)
  test_indexes = test_indexes + testDS.__getIndexesGroups__(task)

  train_dataset = Subset(trainDS, train_indexes)
  test_dataset = Subset(testDS, test_indexes)

  train_loader = DataLoader( train_dataset, num_workers=params.NUM_WORKERS, batch_size=params.BATCH_SIZE)
  test_loader = DataLoader( test_dataset, num_workers=params.NUM_WORKERS, batch_size=params.BATCH_SIZE )

  if(task == 0):
    torch.save(resNet, 'resNet_task{0}.pt'.format(task))
  
  

  utils.trainfunction(task, train_loader, train_splits)
  param = utils.evaluationTest(task, test_loader, test_splits) #evaluate test set at step task
  pars_tasks[int(task/10)] = param #pars_task[i] = (accuracy, loss) at i-th task	

train_splits = trainDS.splits
test_splits = testDS.splits

task = 0 


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


At step  0  and at epoch =  0  the loss is =  0.2949681282043457  and accuracy is =  0.1968
At step  0  and at epoch =  1  the loss is =  0.26361843943595886  and accuracy is =  0.3256
At step  0  and at epoch =  2  the loss is =  0.252512663602829  and accuracy is =  0.3908
At step  0  and at epoch =  3  the loss is =  0.216915562748909  and accuracy is =  0.4324
At step  0  and at epoch =  4  the loss is =  0.17416445910930634  and accuracy is =  0.4846
At step  0  and at epoch =  5  the loss is =  0.16072317957878113  and accuracy is =  0.5212
At step  0  and at epoch =  6  the loss is =  0.1329203099012375  and accuracy is =  0.5496
At step  0  and at epoch =  7  the loss is =  0.13106726109981537  and accuracy is =  0.5984
At step  0  and at epoch =  8  the loss is =  0.11575031280517578  and accuracy is =  0.5956
At step  0  and at epoch =  9  the loss is =  0.08751790970563889  and accuracy is =  0.64
At step  0  and at epoch =  10  the loss is =  0.08377613872289658  and accura

In [0]:
plotTask(pars_tasks)